In [5]:
import fasttext
import fasttext.util
import json
import nltk
from nltk.tokenize import RegexpTokenizer
from snowballstemmer import TurkishStemmer
import numpy as np
from numpy.linalg import norm
# fasttext.util.download_model('tr', if_exists='ignore')

In [6]:
cos_sim = lambda q_vector, vector : np.dot(q_vector, vector)/(norm(q_vector)*norm(vector))

In [8]:
ft = fasttext.load_model(r'cc.tr.300.bin')
STOPWORD_LIST = nltk.corpus.stopwords.words('turkish')
CUSTOMWORD_LIST = ["wifi", "section", "metu", "office"] # custom word list

In [9]:
# preprocessor for quesiton strings to transform strings to word arrays.
# TODO: a better alternative than snowball stemmer should be used. 
def preProcessor(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+') # removes all punctuation
    words = tokenizer.tokenize(sentence)

    ts = TurkishStemmer()
    result = []
    for word in words:
        if word not in STOPWORD_LIST:
            if word in CUSTOMWORD_LIST:
                result.append(word)
            else:
                result.append(ts.stemWord(word))
    return result


In [10]:
# Gets the quesitons dictionary and returns dictionary of dictionaries 
# where subdictionaries consists of keys as each term and values as 
# number of occurencies.
# This version treats all quesitons in a category as a whole document.  
def TF(questions):
    result = {}
    for category in questions:
        result[category] = {}
        for question in questions[category]:
            term_arr = preProcessor(question)
            for term in term_arr:
                if term not in result[category]:
                    result[category][term] = 1
                else:
                    result[category][term] += 1
    return result

def normalizedTF(questions):
    result = {}
    tf = TF(questions)
    for category in tf:
        result[category] = {}
        no_of_terms = len(tf[category])
        for term in tf[category]:
            result[category][term] = tf[category][term] / no_of_terms
    return result

# Calculates inverse document frequency of a term
def IDF(term, questions):
    number_of_documents = len(questions)
    number_of_occurences = 0
    tf = TF(questions)
    for category in tf:
        # number_of_documents += len(tf) # counting all questions, not categories; this can be changed
        if term in tf[category]:
            number_of_occurences+=1
    if number_of_occurences == 0:
        return 0
    return 1 + np.log(number_of_documents / number_of_occurences)


def TFxIDF(terms_string, questions):
    result = {}
    normal_tf = normalizedTF(questions)
    terms = preProcessor(terms_string)

    for category in normal_tf:
        result[category] = {}
        for term in terms:
            idf = IDF(term, questions)
            if term in normal_tf[category]:
                result[category][term] = normal_tf[category][term] * idf
            else:
                result[category][term] = 0
    return result

# returns the most likely category as an heuristic for fasttext classifier
def categoryHeuristic(query, questions):
    tfidf = TFxIDF(query, questions)
    max_val = -1
    max_category = ""
    for category in tfidf:
        for term in tfidf[category]:
            if tfidf[category][term] > max_val:
                max_category = category
                max_val = tfidf[category][term]
    return max_category

In [11]:
f = open("./question_categories.json")
questions = json.load(f)

#print(TF(questions))
#print(IDF("wifi", questions))
print(TFxIDF("wifi şifremi unuttum", questions))
print(categoryHeuristic("wifi şifremi unuttum", questions))

{'sifre': {'wifi': 0, 'şifre': 0, 'unut': 0.1678889830934488}, 'lisanslÄ± yazÄ±lÄ±mlar': {'wifi': 0, 'şifre': 0, 'unut': 0}, 'wifi': {'wifi': 0.5596299436448293, 'şifre': 0, 'unut': 0}}
wifi


In [12]:
def getQuestionVectors(raw_questions):
    question_vectors = {}
    for key in questions:
        question_vectors[key] = []
        for q in questions[key]:
            question_vectors[key].append(ft.get_sentence_vector(q))
    return question_vectors

In [13]:
def questionClassifier(user_question, question_vectors, raw_quesitons):
    q_vector = ft.get_sentence_vector(user_question)

    most_similar_question = ""
    most_similar_category = ""

    max_similarity = 0

    tfidf_category = categoryHeuristic(user_question, raw_quesitons)

    for key in question_vectors:
        for vector, question in zip(question_vectors[key], questions[key]):
            sim = cos_sim(q_vector, vector)
            if sim > max_similarity:
                max_similarity = sim
                most_similar_question = question
                most_similar_category = key

    if most_similar_category != tfidf_category:
        print("--MUHTEMEL YANLIŞ ANLAMALAR SÖZ KONUSU--")
        print("TF.IDF kategori tahmini: %s" % tfidf_category)
        print("FastText kategori tahmini: %s" % most_similar_category)
        # search for the question under the tfidf heuristic category
        most_similar_question = ""
        most_similar_category = ""

        max_similarity = 0

        most_similar_category = tfidf_category
        for vector, question in zip(question_vectors[tfidf_category], questions[tfidf_category]):
            sim = cos_sim(q_vector, vector)
            if sim > max_similarity:
                max_similarity = sim
                most_similar_question = question
        
    else:
        print("Soru kategorisi: %s" % most_similar_category) 
        print("En yakın soru: %s" % most_similar_question)
    
    return (most_similar_category, most_similar_question)

In [14]:
f = open("./answers.json")
answers = json.load(f)

def getAnswer(user_question, question_vectors, raw_quesitons):
    category,question = questionClassifier(user_question, question_vectors, raw_quesitons)
    ans = answers[category]
    return ans

In [15]:
question_vectors = getQuestionVectors(questions)
questionClassifier("wifi şifremi unuttum", question_vectors, questions)

print("cevap: " + getAnswer("wifi şifremi unuttum", question_vectors, questions))

--MUHTEMEL YANLIŞ ANLAMALAR SÖZ KONUSU--
TF.IDF kategori tahmini: wifi
FastText kategori tahmini: sifre
--MUHTEMEL YANLIŞ ANLAMALAR SÖZ KONUSU--
TF.IDF kategori tahmini: wifi
FastText kategori tahmini: sifre
cevap: Detalied information can be found at https://faq.cc.metu.edu.tr/groups/wireless-network
